# Modelagem de Tópicos com LDA

In [1]:
import pandas as pd
import numpy as np
import altair as alt
from sklearn.decomposition import LatentDirichletAllocation

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

import re
from time import time

[nltk_data] Downloading package stopwords to /home/mateus/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [3]:
rock_lyrics = pd.read_csv('https://raw.githubusercontent.com/nazareno/palavras-nas-letras/master/letras-ptbr-rock-grande.csv')

rock_lyrics['titulo_limpo'] = rock_lyrics['SName'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')
rock_lyrics.drop_duplicates(subset ="titulo_limpo", 
                       keep = 'first', inplace = True)

count = rock_lyrics['Lyric'].str.split().str.len()
rock_lyrics = rock_lyrics[count > 50]

print('Letras: {}'.format(rock_lyrics.shape[0]))
print('\nPor artista: ')
rock_lyrics['Artist'].value_counts()[0:20]

Letras: 5389

Por artista: 


Rita Lee                   245
Erasmo Carlos              206
Lulu Santos                202
Engenheiros do Hawaii      178
Charlie Brown Jr           177
Titãs                      174
Capital Inicial            174
Raul Seixas                169
Os Paralamas do Sucesso    161
Biquini Cavadão            157
Barão Vermelho             144
Velhas Virgens             131
Fresno                     129
Jota Quest                 125
Skank                      122
Ira!                       119
Pato Fu                    118
Rosa de Saron              112
Detonautas                 108
PG                         108
Name: Artist, dtype: int64

#### Observações iniciais
Vamos começar realizando algumas observações superficiais do Dataframe, a partir dos métodos sample(), info() e value_counts() de artistas.

In [4]:
rock_lyrics.sample(10)

,SName,Lyric,Artist,Songs,Popularity,Genre,Genres,titulo_limpo
3958,Continuar,Abro os meus olhos já é de manhã. À noite é me...,Oficina G3,134,2.3,Rock,Hard Rock; Pop/Rock; Rock; Gospel/Religioso; H...,Continuar
6359,Eu Me Amo,Há tanto tempo eu vinha me procurando. Quanto ...,Ultraje A Rigor,90,2.1,Rock,Rock; Pop/Rock; Rock Alternativo; Punk Rock; M...,Eu Me Amo
3213,Auto-estima,Eu falo com você. Até quando você não está. E ...,Lulu Santos,271,10.0,Rock,Pop/Rock; Rock; Pop; MPB; Dance; Electronica; ...,Auto-estima
1781,Kamasutra,"Hu, hu, hu. Hu, hu, hu. Hu, hu, hu. Em que pos...",Erasmo Carlos,225,2.3,Rock,Rock; Jovem Guarda; Romântico; MPB; Soul Music...,Kamasutra
5599,Latas Retorcidas,Correndo em busca de emoções. Em alta velocida...,Rosa de Saron,134,8.2,Rock,Gospel/Religioso; Rock; Pop/Rock; Romântico; R...,Latas Retorcidas
299,A Saudade é o Museu do Amor,Nuvens deslizam pelo céu. Cobrindo estrelas co...,Biquini Cavadão,162,3.4,Rock,Rock; Pop/Rock; Romântico; Punk Rock; New Wave...,A Saudade e o Museu do Amor
1550,Twisting By The Pool (tradiução),Nós estamos indos passar férias agora. Indo fi...,Dire Straits,64,5.0,Rock,Soft Rock; Rock; Blues; Rock Alternativo; Coun...,Twisting By The Pool (tradiucao)
3155,Romeu E Julieta,Romeu e Julieta. Assim que o amor entrou no me...,Los Hermanos,80,5.5,Rock,Rock Alternativo; Rock; MPB; Romântico; Indie;...,Romeu E Julieta
1595,A Fábula,"Era uma vez um planeta mecânico,. lógico, onde...",Engenheiros do Hawaii,193,11.9,Rock,Pop/Rock; Rock; Pop; Rock Alternativo; MPB; Se...,A Fabula
1981,Sexo e Humor,Sexo e humor na rua do Ouvidor. Na noite fútil...,Erasmo Carlos,225,2.3,Rock,Rock; Jovem Guarda; Romântico; MPB; Soul Music...,Sexo e Humor


In [5]:
rock_lyrics.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5389 entries, 0 to 6726
Data columns (total 8 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   SName         5389 non-null   object 
 1   Lyric         5389 non-null   object 
 2   Artist        5389 non-null   object 
 3   Songs         5389 non-null   int64  
 4   Popularity    5389 non-null   float64
 5   Genre         5389 non-null   object 
 6   Genres        5389 non-null   object 
 7   titulo_limpo  5389 non-null   object 
dtypes: float64(1), int64(1), object(6)
memory usage: 378.9+ KB


In [6]:
rock_lyrics['Artist'].value_counts()[0:20]

Rita Lee                   245
Erasmo Carlos              206
Lulu Santos                202
Engenheiros do Hawaii      178
Charlie Brown Jr           177
Titãs                      174
Capital Inicial            174
Raul Seixas                169
Os Paralamas do Sucesso    161
Biquini Cavadão            157
Barão Vermelho             144
Velhas Virgens             131
Fresno                     129
Jota Quest                 125
Skank                      122
Ira!                       119
Pato Fu                    118
Rosa de Saron              112
Detonautas                 108
PG                         108
Name: Artist, dtype: int64

#### Pré-processamento
Faremos o pré-processamento dos textos das letras a partir da exclusão de caracteres especiais

In [7]:
stop_words = set(stopwords.words("portuguese"))
stop_words.update(['que', 'até', 'esse', 
                    'essa', 'pro', 'pra',
                    'oi', 'lá', 'blá', 'bb', 
                    'bbm', 'abm', 'cbm', 
                    'dbm', 'dos', 
                    'ltda', 'editora']), 
print(len(stop_words))

216


In [8]:
rock_lyrics_c = []

for i in range(len(rock_lyrics.Lyric)):
    lyric = rock_lyrics['Lyric'].iloc[i]

    lyric  = re.sub("(\\d|\\W)+|\w*\d\w*"," ",lyric )
    lyric = ' '.join(s for s in lyric.split() if (not any(c.isdigit() for c in s)) and len(s) > 2)
    rock_lyrics_c.append(lyric)

rock_lyrics_c[5:7]

['Saudade Espero que logo logo vai passar Vontade Hoje quero matar com você Sempre que puder voltarei aqui Sempre que puder quero ver sorrir Felicidade vale tiver alguém pra dividir vou levar pra ver sol pra ver mar Andar meu caminho levo emoção pra acelerar meu coração Não quero estar sozinho vou levar pra ver sol pra ver mar Andar meu caminho levo emoção pra acelerar meu coração levo comigo aprendi usar solidão vivo bem comigo mesmo então Respeito com liberdade Onde estiver seja bem vinda agora você quiser Sempre que puder voltarei aqui Sempre que puder quero ver sorrir Felicidade vale tiver alguém pra dividir vou levar pra ver sol pra ver mar Andar meu caminho levo emoção pra acelerar meu coração Não quero estar sozinho vou levar pra ver sol pra ver mar Andar meu caminho levo emoção pra acelerar meu coração levo comigo vou levar pra ver sol pra ver mar Andar meu caminho levo emoção pra acelerar meu coração Não quero estar sozinho vou levar pra ver sol pra ver mar Andar meu caminho l

#### Criando vetores TF

In [9]:
#TF-IDF vectorizer
tfv_lyrics = CountVectorizer(
        min_df = 5,
        max_df = 0.3,
        max_features = 10000,
        stop_words = stop_words, 
        ngram_range = (1,2)
  )


#transform
vec_lyrics = tfv_lyrics.fit_transform(rock_lyrics_c)

#returns a list of words.
lyrics_words = tfv_lyrics.get_feature_names()

print(len(lyrics_words), vec_lyrics.shape)

10000 (5389, 10000)


/home/mateus/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


### Escolhendo o número de componentes
Utilizaremos o otimizador da classe GridSearchCV para buscar qual o melhor valor de n_components

In [10]:
from sklearn.model_selection import GridSearchCV
from sklearn.decomposition import LatentDirichletAllocation

In [11]:
search_params = {'n_components': [5, 10, 15, 20]}
lda = LatentDirichletAllocation()
model = GridSearchCV(lda, param_grid=search_params)
model.fit(vec_lyrics)

GridSearchCV(estimator=LatentDirichletAllocation(),
             param_grid={'n_components': [5, 10, 15, 20]})

In [13]:
# Best model
best_lda_model = model.best_estimator_

# Model Parameters
print("Best Model's Params: ", model.best_params_)

# Log Likelihood Score
print("Best Log Likelihood Score: ", model.best_score_)

# Perplexity
print("Model Perplexity: ", best_lda_model.perplexity(vec_lyrics))

Best Model's Params:  {'n_components': 5}
Best Log Likelihood Score:  -802099.1661659401
Model Perplexity:  2826.316089969204


### Encontrando tópicos

In [14]:
lda = LatentDirichletAllocation(n_components=5, 
                                learning_method='batch', # 'online' equivale a minibatch no k-means
                                random_state=0)

t0 = time()

lda.fit(vec_lyrics)
doc_topic_matrix = lda.transform(vec_lyrics)

print("done in %0.3fs." % (time() - t0))


done in 23.172s.


In [15]:
print('Matriz documento-tópicos:' + str(doc_topic_matrix.shape))
print('Matriz tópicos-termos:' + str(lda.components_.shape))

Matriz documento-tópicos:(5389, 5)
Matriz tópicos-termos:(5, 10000)


In [16]:
def print_top_words(model, feature_names, n_top_words):
  for topic_idx, topic in enumerate(model.components_):
    print("\n--\nTopic #{}: ".format(topic_idx + 1))
    message = ", ".join([feature_names[i]
                          for i in topic.argsort()[:-n_top_words - 1:-1]])
    print(message)
  print()

def display_topics(W, H, feature_names, documents, no_top_words, no_top_documents):
    for topic_idx, topic in enumerate(H):
        print("\n--\nTopic #{}: ".format(topic_idx + 1))
        print(", ".join([feature_names[i]
                for i in topic.argsort()[:-no_top_words - 1:-1]]).upper())
        top_d_idx = np.argsort(W[:,topic_idx])[::-1][0:no_top_documents]
        for d in top_d_idx: 
          doc_data = rock_lyrics[['Artist', 'SName']].iloc[d]
          print('{} - {} : \t{:.2f}'.format(doc_data[1], doc_data[0], W[d, topic_idx]))

In [17]:
# Imprimindo palavras mais populares de cada tópico
print_top_words(lda, lyrics_words, 20)


--
Topic #1: 
vou, gente, vida, bem, quer, vem, onde, ninguém, porque, mundo, medo, dia, tão, amor, ser, hoje, ficar, mão, todo, mim

--
Topic #2: 
mundo, vida, todo, ser, you, ninguém, aqui, todo mundo, the, yeah, porque, cada, então, sempre, quero, toda, tempo, deus, tão, ter

--
Topic #3: 
mim, sei, quero, ser, vou, agora, ver, aqui, tempo, sempre, dia, bem, vida, assim, viver, nunca, ter, tão, gente, lugar

--
Topic #4: 
amor, vida, coração, tempo, deus, tão, sempre, faz, luz, bem, ser, vem, sol, tanto, quero, céu, dia, sei, dizer, assim

--
Topic #5: 
quero, nada, dia, ser, pode, ver, noite, nunca, mulher, todos, todo, faz, saber, aqui, homem, vida, sempre, sei, vão, bem



In [18]:
# Mostrando as letras que 'melhor' pertencem a cada tópico
display_topics(doc_topic_matrix,
               lda.components_, 
               lyrics_words,
               rock_lyrics,
               15, 
               10)


--
Topic #1: 
VOU, GENTE, VIDA, BEM, QUER, VEM, ONDE, NINGUÉM, PORQUE, MUNDO, MEDO, DIA, TÃO, AMOR, SER
A Todas As Comunidades do Engenho Novo - O Rappa : 	1.00
Reggae Town (Feat. Natiruts) - Jota Quest : 	1.00
Reggae Town - Jota Quest : 	1.00
Blecaute (Feat. Anitta, Nile Rodgers) - Jota Quest : 	1.00
Comida - Titãs : 	1.00
Bunda Boa - Velhas Virgens : 	1.00
100 Critério - Detonautas : 	1.00
F.d.p - Velhas Virgens : 	1.00
A Minha Rádio É Rock - CPM 22 : 	1.00
Posso Perder Minha Mulher, Minha Mãe, Desde Que Eu Tenha O Rock And Roll - Mutantes : 	1.00

--
Topic #2: 
MUNDO, VIDA, TODO, SER, YOU, NINGUÉM, AQUI, TODO MUNDO, THE, YEAH, PORQUE, CADA, ENTÃO, SEMPRE, QUERO
Inimigo Invisível (part: Kurupt, Pmc, Marcelo D2, Mi) - Nx Zero : 	1.00
Bem Ou Mal (Part: Marcelo Mancini, Aggro Santos, Kamau) - Nx Zero : 	1.00
Reza Vela / Norte-Nordeste Me Veste - O Rappa : 	1.00
Vivendo A Vida Numa Louca Viagem - Charlie Brown Jr : 	1.00
Homenagem ao Santos Futebol Clube - Charlie Brown Jr : 	1.00
Ex-Qu

In [19]:
# Exemplos de letras muito associadas com um tópico

main_topic = []
mt_prob = []
for l in range(len(rock_lyrics['Artist'])):
  main_topic.append(doc_topic_matrix[l,:].argmax() + 1)
  mt_prob.append(doc_topic_matrix[l,:].max())

rock_lyrics = rock_lyrics.assign(main_topic = main_topic, main_topic_prob = mt_prob)

In [31]:
topico = 2
pd.options.display.max_colwidth = 300
rock_lyrics[rock_lyrics['main_topic'] == topico].sort_values('main_topic_prob', ascending = False)[['main_topic_prob','Artist', 'Lyric']].head(20).sample(10)

/home/mateus/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,main_topic_prob,Artist,Lyric
3934,0.997109,O Rappa,"Minhas irmãs, meus irmãos. se assumam como realmente são. Não deixem que suas matrizes. que suas raízes morram por falta de irrigação. Ser nortista e nordestino meus conterrâneos. num é ser seco nem litorâneo. É ter em nossas mãos um destino. nunca clandestino para os desfechos metropolitanos"". ..."
6275,0.993287,Titãs,"Intro: F#m7. (F#m7). Miséria é miséria em qualquer canto. Riquezas são diferentes. Índio, mulato, preto, branco. Miséria é miséria em qualquer canto. Riquezas são diferentes. Miséria é miséria em qualquer canto. Filhos, amigos, amantes, parentes. Riquezas são diferentes. Ninguém sabe falar esper..."
266,0.993684,Ben Harper,"É só isso. Não tem mais jeito. Acabou, boa sorte. Não tenho o que dizer. São só palavras. E o que eu sinto. Não mudará. Tudo o que quer me dar. É demais. É pesado. Não há paz. Tudo o que quer de mim. Irreais. Expectativas. Desleais. That's it. There is no way. It's over, good luck. I have nothin..."
1693,0.995693,Engenheiros do Hawaii,. E. todo mundo é eterno. D. todo mundo é moderno. A B. como um relógio antigo. E. no underground. D. no mainstream. A. todo mundo é moderno. B. todo mundo é eterno. E. ontem. E4. ano passado. E E4. antigamente. E. amanhã. E4. ano que vem. E. ano dois mil. A D/A. todo mmundo é moderno. A D/A. to...
3746,0.997237,Nx Zero,"Vamos Vamos cancela seus planos,. Viva a vida like you a ""sixteen"" anos.. ""Experiences"", a part tantos, tantos. Nx zero e o Aggro Santos.. Bem e o Mal, the Good and bad, Sista'!. Baixo astral, jus' Make me sad.. A never listen to comply, Aggro up on fight. Rockstar lifestyle living as I like.. T..."
292,0.995228,Biquini Cavadão,Eu presto atenção. No que eles dizem. Mas eles não dizem nada. Yeah! Yeah!. Fidel e Pinochet. Tiram sarro de você. Que não faz nada. Yeah! Yeah!. E eu começo achar normal. Que algum boçal. Atire bombas na embaixada. Yeah! Yeah! Oh! Oh!.... . Se tudo passa. Talvez você passe por aqui. E me faça. ...
3886,0.993748,O Rappa,"Não falta a marca da crise. O pé incomoda, incomoda com o furo. Eu quase encosto, encosto no muro. No lado de cá, no lado de cá da vitória. No escuro, o teto é a laje. Acende e apaga, apaga a fogueira. No charco molhado de papelão. Coberto de fogo da brasa da fogueira. Na brasa da fogueira (3x)...."
6321,0.993525,Titãs,Dizem que guardam. Um bom lugar pra mim no céu. Logo que eu for pro beleléu. A minha vida só eu sei como guiar. Pois ninguém vai me ouvir se eu chorar. Mas enquanto o sol puder arder. Não vou querer meus olhos escurecer. Pois se eles querem meu sangue. Verão o meu sangue só no fim. Se eles quere...
5749,0.995158,Santana,Dar um jeito (oh oh oh). Dar um jeito (oh oh oh). When you got nothing to eat. And you find it hard to sleep. How you struggle on the streets. And you can't escape the heat. But your heart could feel that beat. So you get up off your feet. Ain't no mountain you can't reach. Grab a star and make ...
6370,0.993996,Ultraje A Rigor,"(Dinheiro, Dinheiro, Dinheiro, Dinheiro). Mim quer tocar,. Mim gosta ganhar dinheiro (dinheiro). Me want to play,. Me love to get the money (money). Mim é brasileiro,. Mim gosta banana (banana). Mas mim também quer votar. Mim também quer ser bacana (bacana). Mim quer tocar,. Mim gosta ganhar din..."


In [21]:
# O quanto determinada música pertence a cada tópico
topicnames = ["Topic {}".format(i + 1) for i in range(doc_topic_matrix.shape[1])]

lyric_topic = pd.DataFrame(np.round(doc_topic_matrix, 2), columns=topicnames, index = rock_lyrics.index)
lyric_topic[['Artist', 'SName']] = rock_lyrics[['Artist','SName']]

ordem = ['Artist', 'SName']
ordem.extend(topicnames)
lyric_topic = lyric_topic[ordem]

lyric_topic.query('Artist == "Titãs"').sort_values(by = 'Topic 4', ascending = False).head()

,Artist,SName,Topic 1,Topic 2,Topic 3,Topic 4,Topic 5
6315,Titãs,Provas de Amor,0.00,0.00,0.00,0.99,0.00
6179,Titãs,Alma Lavada,0.00,0.00,0.00,0.99,0.00
6220,Titãs,Doze Flores Amarelas,0.00,0.00,0.00,0.98,0.00
6156,Titãs,O Pulso,0.00,0.00,0.00,0.98,0.00
6168,Titãs,A Face do Destruidor,0.01,0.01,0.01,0.95,0.01


### Visualizando os tópicos

In [22]:
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.sklearn
pyLDAvis.enable_notebook()

In [23]:
pyLDAvis.sklearn.prepare(lda, vec_lyrics, tfv_lyrics, sort_topics=False, mds = 'tsne')

/home/mateus/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/mateus/anaconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)
/home/mateus/anaconda3/lib/python3.8/site-packages/pyLDAvis/_prepare.py:246: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  default_term_info = default_term_info.sort_values(
/home/mateus/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
0      150.222153  -72.269554       1        1  15.725220
1       51.163364 -159.968018       2        1  12.955926
2      -73.144554   21.523815       3        1  30.645132
3      -66.410133 -110.606438       4        1  21.093289
4       54.417324   12.443444       5        1  19.580433, topic_info=         Term         Freq        Total Category  logprob  loglift
9768      vou  4523.000000  4523.000000  Default  30.0000  30.0000
438      amor  3371.000000  3371.000000  Default  29.0000  29.0000
4146    gente  2061.000000  2061.000000  Default  28.0000  28.0000
5641    mundo  2439.000000  2439.000000  Default  27.0000  27.0000
2642     deus  1168.000000  1168.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
9768      vou   321.849233  4523.840796   Topic5  -5.6349  -1.0124
5641    mundo   285.297340  2439.409883   Topic5  -5.7555  -0.5153
763     assim   269.971349  1887.353408   Topic5  -5.8107  -0.3140
5884  ninguém   253.421777  1626.560466   Topic5  -5.8739  -0.2285
191     agora   250.008334  1862.431055   Topic5  -5.8875  -0.3775

[448 rows x 6 columns], token_table=      Topic      Freq            Term
term                                 
160       4  0.980495          adorar
191       1  0.065506           agora
191       2  0.068190           agora
191       3  0.644319           agora
191       4  0.087520           agora
...     ...       ...             ...
9951      2  0.998062             you
9958      2  0.992380            your
9965      2  0.056117             zum
9965      5  0.925923             zum
9982      3  0.971990  óculos escuros

[981 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 2, 3, 4, 5])

In [24]:
from sklearn.manifold import TSNE

lyrics_embedded = TSNE(n_components=2, verbose=1, perplexity=40, early_exaggeration=20).fit_transform(doc_topic_matrix)

/home/mateus/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/mateus/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:780: FutureWarning: The default initialization in TSNE will change from 'random' to 'pca' in 1.2.
  warnings.warn(
/home/mateus/anaconda3/lib/python3.8/site-packages/sklearn/manifold/_t_sne.py:790: FutureWarning: The default learning rate in TSNE will change from 200.0 to 'auto' in 1.2.
  warnings.warn(


[t-SNE] Computing 121 nearest neighbors...
[t-SNE] Indexed 5389 samples in 0.005s...
[t-SNE] Computed neighbors for 5389 samples in 0.220s...
[t-SNE] Computed conditional probabilities for sample 1000 / 5389
[t-SNE] Computed conditional probabilities for sample 2000 / 5389
[t-SNE] Computed conditional probabilities for sample 3000 / 5389
[t-SNE] Computed conditional probabilities for sample 4000 / 5389
[t-SNE] Computed conditional probabilities for sample 5000 / 5389
[t-SNE] Computed conditional probabilities for sample 5389 / 5389
[t-SNE] Mean sigma: 0.003024
[t-SNE] KL divergence after 250 iterations with early exaggeration: 133.195160
[t-SNE] KL divergence after 1000 iterations: 0.728884


In [25]:
rock_lyrics = rock_lyrics.assign(tsne1 = lyrics_embedded[:,0], tsne2 = lyrics_embedded[:,1])

alt.Chart(rock_lyrics.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2', 
    color = 'main_topic:N',
    size = 'main_topic_prob',
    tooltip = ['Artist', 'SName', 'main_topic', 'main_topic_prob']
).interactive()

/home/mateus/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


alt.Chart(...)

### Agrupamento segundo os tópicos

In [41]:
from sklearn.cluster import KMeans

kmeans =  KMeans(n_clusters=5, random_state=0)
kmeans.fit(doc_topic_matrix)
# An exception is thrown by this code (for some reason), but the labels are successfully generated
labels = kmeans.predict(doc_topic_matrix)
labels

/home/mateus/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


AttributeError: 'NoneType' object has no attribute 'split'

array([3, 0, 2, ..., 2, 3, 2], dtype=int32)

In [42]:
lyrics2 = lyric_topic.assign(grupo = labels, 
                              tsne1 = lyrics_embedded[:,0], tsne2 = lyrics_embedded[:,1])

alt.Chart(lyrics2.sample(500)).mark_circle(
    opacity = .7,
    size = 30
).encode(
    x = 'tsne1',
    y = 'tsne2', 
    color = 'grupo:N',
    tooltip = [str(c) for c in lyrics2.columns]
).interactive()

/home/mateus/anaconda3/lib/python3.8/site-packages/ipykernel/ipkernel.py:287: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)
/home/mateus/anaconda3/lib/python3.8/asyncio/events.py:81: DeprecationWarning: `run_cell_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  self._context.run(self._callback, *self._args)


alt.Chart(...)